In [ ]:
import ffmpeg  # Library to work with audio and video files, here used to extract audio
import subprocess  # To make subprocess call using terminal
import os
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
# Extracts audio from video
# ffmpeg -i input_filename -ab bitrate -ar frequency -vn output_filename

# TODO: Change input_filename.mp4 to file to be converted
# If your output is '127' that means command not found and likely ffmpeg has not been correctly installed.
command = 'ffmpeg -i input_filename.mp4 -ab 160k -ar 44100 -vn output_filename.wav'
subprocess.call(command, shell=True)

In [ ]:
# TODO: Insert the api_key and url as strings as listed on your IBM Cloud account (Watson Speech to Text service)
api_key = ''
url = ''

In [ ]:
# Setups service
authenticator = IAMAuthenticator(api_key)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

In [ ]:
# Compresses audio
command = 'ffmpeg -i output_filename.wav -vn -ar 44100 -ac 2 -b:a 192k output_filename.mp3'
subprocess.call(command, shell=True)

In [ ]:
# Splits audio into smaller and more manageable files
command = 'ffmpeg -i output_filename.mp3 -f segment -segment_time 1800 -c copy audio_file_%03d.mp3'
subprocess.call(command, shell=True)

In [ ]:
files = []
for filename in os.listdir('.'):
    if filename.startswith('audio_file_') and filename.endswith('.mp3'):
        files.append(filename)
files.sort()
files

In [ ]:
# Model list: https://cloud.ibm.com/apidocs/speech-to-text#listmodels
# Japanese models: ja-JP_BroadbandModel, ja-JP_NarrowbandModel
# JP_Broadband seems to capture the speech more accurately
results = []
for filename in files:
    with open(filename, 'rb') as f:
        res = stt.recognize(audio=f, content_type= 'audio/mp3', model='ja-JP_BroadbandModel', continuous=True,  
                           inactivity_timeout=1200).get_result()
        results.append(res)

In [ ]:
results[0]  # Shows speech to text results for the first audio file

In [ ]:
text = []
for file in results:
    for result in file['results']:
        text.append(result['alternatives'][0]['transcript'].rstrip() + '.\n')

In [ ]:
transcript_list = ''.join(text).split()

for n, i in enumerate(transcript_list):
    if i.endswith(u'.'):
        transcript_list[n] = i[:-1]

for n, i in enumerate(transcript_list):
    # Light text cleaning below
    if i == u'ですね':
        transcript_list[n] = u'ですね。'
    if i == u'はい':
        transcript_list[n] = u'はい、'
    if i.endswith(u'ます'):
        transcript_list[n] = i.replace(u'ます', u'ます。')
    if i.endswith(u'ましょう'):
        transcript_list[n] = i.replace(u'ましょう', u'ましょう。')
    if i == u'ね' and transcript_list[n-1] == u'です':
        transcript_list[n] = u'ね。'
    if i == u'ねえ':
        transcript_list[n] = u'ね。'
    if i.startswith('D_'):  # Dropoffs in speech. Often in places like ええと and まあー
        transcript_list[n] = ''

In [ ]:
transcript = ''.join(transcript_list)
transcript = transcript.replace(u'ます。ので', 'ますので、')
transcript

In [ ]:
# Saves transcript variable as 'トランスクリプト_long.txt'
with open('トランスクリプト_long.txt', 'w') as out:
    out.writelines(transcript)